<a href="https://colab.research.google.com/github/ebamberg/research-projects-ml/blob/main/agents_and_routing/examples_agents_with_critique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install ollama langchain_community --quiet

host="localhost:11434"
modelid="llama3.2"
emb_modelid="mxbai-embed-large"

get_ipython().system_raw("curl -fsSL https://ollama.com/install.sh | sh")
get_ipython().system_raw("ollama serve &")
get_ipython().system_raw(f"ollama pull {modelid}")
get_ipython().system_raw(f"ollama pull {emb_modelid}")


In [10]:
!pip install openai --quiet

In [11]:
get_ipython().system_raw(f"ollama pull {modelid}")

In [12]:
from openai import OpenAI


llm = OpenAI(
        base_url=f"http://{host}/v1",
        api_key="ollama",  # required, but unused
    )




In [13]:
def call(system_prompt: str, message: str) -> str:
  completion = llm.chat.completions.create(
      model=modelid,
      messages=[
          {"role": "system", "content": system_prompt},
          {
              "role": "user",
              "content": message,
          },
      ],
  )

  return completion.choices[0].message.content

In [14]:
def marketing_campaign_agent(product : str):
  """
     <PRODUCT>{product}</PRODUCT>

     Your task is to create an compelling marketing campaign for the product provided between the <product> tag.
  """
  return call ("your are a marketing campaign export.", marketing_campaign_agent.__doc__.format(product=product) )

In [18]:
def ready_validation_agent(marketing_text : str) -> bool:
  """

    <CONTENT>{text}</CONTENT>
    I have asked you to create a marketing text for a product  earlier. You can find the text between the <CONTENT> tags.

    Is this the final version of the marketing text and ready to publish or can we improve it?

     Your task is to answer with
     - TRUE if this is the final version of the marketing text and ready to publish.
     - FALSE if this is not the final version of the marketing text and we need to improve it.

     Only answer with TRUE or FALSE. add nothing else to your response.

    """
  result = call ("your are a marketing campaign export.", ready_validation_agent.__doc__.format(text=marketing_text) )
  print (result)
  return result.strip().lower() == "true"

In [31]:
def refinements_agent(product : str, marketing_text : str) -> str:
  """
  <PRODUCT>{product}</PRODUCT>

  <CONTENT>{text}</CONTENT>

  I have asked you to create a marketing text for a product earlier. You can find the text between the <CONTENT> tags.

  This marketing text is not final and not yet ready for publishing.

  Your task is to refine the marketing text provided betwen the <CONTENT> tag with the follwing refinements. Don't use any other sources.

  ### OUTPUT
  Please output a  list of improvements you have found.

  """
  return call ("your are a marketing campaign export.", refinements_agent.__doc__.format(product=product, text=marketing_text) )

In [42]:
def refine_agent(product : str, marketing_text : str, refinements : str ) -> str:
  """
  <PRODUCT>{product}</PRODUCT>

  <CONTENT>{text}</CONTENT>

  <REFINE>{refinements}</REFINE>

  I have asked you to create a marketing campaign for a product earlier. You can find the marketing campaign between the <CONTENT> tags.

  Your task is to add the refinements provided between <REFINE> tags to that marketing campaign .



  """
  return call ("your are a marketing campaign export.", refinements_agent.__doc__.format(product=product, text=marketing_text, refinements=refinements) )

In [48]:
product = "iPhone 15"

iteration=1
marketing_text = marketing_campaign_agent(product)
while not ready_validation_agent(marketing_text) and iteration<10:
  print (f"ITERATION: {iteration}" )
  refinements = refinements_agent(product, marketing_text)
  next_text_iteration = refine_agent(product, marketing_text, refinements)
  print (refinements)
  iteration+=1
  marketing_text = next_text_iteration


FALSE
ITERATION: 1
### Refinement Suggestions:

1. **Campaign Title:** Instead of using a single phrase "Unleash the Future: Introducing the iPhone 15", consider breaking it down into two separate titles:
	* "**Unleash Your Potential**: A New Era in Mobile Technology" (main campaign title)
	* "**Introducing the iPhone 15**: Revolutionizing Smartphones" (sub title)

2. **Campaign Overview:**
	* Consider rephrasing the first sentence to be more concise and attention-grabbing:
	* "The iPhone 15 marks a major milestone in mobile innovation, setting a new standard for performance, design, and user experience."
	* Remove unnecessary words like "represent" and "offer".
3. **Target Audience:**
	* Instead of listing multiple demographics, consider grouping them into more specific categories (e.g., age ranges, job titles).
4. **Key Messages:**
	* Consider rephrasing or condensing some of the bullet points to make it easier to read:
		+ "Unlock your creativity with advanced camera capabilities, A

In [49]:
print (next_text_iteration)


### Refinement Improvements for iPhone 15 Marketing Text

Based on the provided content, I've identified the following refinement suggestions:

1. **Clarify the Tagline**:
   - Original text: "Revolutionize Your Mobile Experience with iPhone 15"
   - Suggested revision: "Experience the Future of Mobile: Introducing iPhone 15"
   The existing tagline is concise but may not fully encapsulate the new device's features.

2. **Enhance the Campaign Overview**:
   - Original text: "Experience the future of mobile revolution with iPhone 15. Boosted performance, enhanced camera capabilities, and AI-driven features make this device a game-changer in your daily life."
   - Suggested revision: 
     * Enhance clarity by breaking up lengthy sentences.
     * Emphasize benefits through words like 'revolutionize' or 'transform.'
     - Revised campaign overview:
       "Unleash the Power of iPhone 15. Boosted performance, AI-driven camera capabilities, and intelligent features redefine your mobile ex

In [45]:
print (marketing_text)
print ("###### critique")
print (is_finished)
print (refinements)
print ("###### next text iteration")
print (next_text_iteration)

**Marketing Campaign: "Empower Your World" - iPhone 15 Launch**

**Campaign Objective:** Drive awareness, generate buzz, and drive sales of iPhone 15 among tech-savvy consumers worldwide.

**Target Audience:** Tech enthusiasts, mobile device users, and individuals looking for a premium smartphone experience.

**Key Messaging:**

* Introducing the latest innovation from Apple - iPhone 15, redefining the smartphone experience with cutting-edge technology and sleek design.
* Unlock your world with iPhone 15, featuring groundbreaking camera capabilities, lightning-fast performance, and an immersive display.

**Marketing Strategies:**

1. **Teaser Campaign:**
	* Release a series of cryptic ads on social media platforms (Instagram, TikTok, Facebook) showcasing a fragmented smartphone interface, hinting at the iPhone 15's advanced features.
	* Create a hashtag (#EmpowerYourWorld) and encourage users to share their own mobile moments, using the iPhone 15 as their device of choice.
2. **Influen